In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install d2l==0.17.6

In [ ]:
import os
path = '/content/drive/MyDrive'
os.chdir(path)

!source venv_d2l/bin/activate

path = '/content/drive/MyDrive/d2l-zh'
os.chdir(path)

In [ ]:
# 自定义层
# 构造一个没有任何参数的自定义层

# 之前我们讲过怎么自定义神经网络和参数，现在我们再往下一点，怎么自定义层
# 和自定义网络没什么太大区别，因为层也是nn.Module的一个子类
import torch
import torch.nn.functional as F
from torch import nn

# CenteredLayer没有参数
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    # 输入减掉均值，使得均值变为0
    def forward(self, X):
        return X - X.mean()

In [ ]:
# 定义一个类的实例，然后把tensor放进去
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [ ]:
# 将层作为组件合并到构建更复杂的模型中
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [ ]:
Y = net(torch.rand(4, 8))
Y.mean()
# 不会真的等于0，有点计算的误差

tensor(6.5193e-09, grad_fn=<MeanBackward0>)

In [ ]:
# 带参数的图层
# 之前我们的参数都是Parameter的实例，所以参数要调用相应的parameter
# 定义一个我们自己的线性层，需要一个输入和一个输出的维度in_units, units
# 然后我们有两个参数，一个是输入大小x输出大小的矩阵，我们用的最简单的随机数，
# 用正一负一直接的均匀分布rand做初始化 -- 这里是正态分布randn做初始化
# bias我们一般用zero，这里是randn，初始化后放进parameter里面

class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    # 通过.data拿到对应的参数
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear) #直接把relu做了

In [ ]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.7225, -1.1366, -0.8056],
        [ 0.0968, -1.8113,  1.1169],
        [ 0.4009, -0.0466,  0.4319],
        [ 0.5123,  0.9666,  0.1310],
        [-1.2404,  0.3156, -0.2553]], requires_grad=True)

In [ ]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.2417, 0.0000]])

In [ ]:
# 使用自定义层直接执行正向传播计算
dense(torch.rand(2, 5))

In [ ]:
# 使用自定义层构建模型
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

In [ ]:
# 读写文件
# 加载和保存张量

import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [ ]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [ ]:
# 可以存一个list
# 存储一个张量列表，然后把它们读回内存
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [ ]:
# 写入或读取从字符串映射到张量的字典
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [ ]:
# 加载和保存模型参数
# 对于一个神经网络，要存什么东西
# pytorch因为是imperative的话，不方便把整个网络的模型存下来，有一定局限性的
# 我们只需要把他的权重存下来就行
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [ ]:
# 将模型的参数存储为一个叫做mlpparms的字典文件
torch.save(net.state_dict(), 'mlp.params')

In [ ]:
# 实例化了原始多层感知机模型的一个备份，直接读取文件中存储的参数
# 重新实现感知机模型，不仅需要把存的params文件带走，还需要把MLP()这个定义带走

# 首先要申明MLP，其中参数已经随机初始化了
# 通过load_state_dict把从文件里读出来的字典放进去
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [ ]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [ ]:
# GPU
!nvidia-smi

Sat Apr 20 06:07:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# 计算设备
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [ ]:
# 查询可用gpu的数量
torch.cuda.device_count()

1

In [ ]:
# 这两个函数允许我们在请求的GPU不存在的情况下运行代码
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [ ]:
# 查询张量所在的设备
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [ ]:
# 存储在GPU上
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [ ]:
# 第二个GPU上创建一个随机张量
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.8839, 0.6231, 0.5262],
        [0.3979, 0.6102, 0.0955]])

In [ ]:
# 要计算X + Y, 我们需要决定在哪里执行这个操作
Z = X.cuda(1)
print(X)
print(Z)

In [ ]:
# 现在数据在同一个GPU上(Z和Y都在)，我们可以将它们相加
# CPU和GPU互相挪的话性能很慢
Y + Z

In [ ]:
Z.cuda(1) is Z

In [ ]:
# 神经网络于GPU
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [ ]:
net(X)

tensor([[1.0077],
        [1.0077]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
# 确认模型参数存储在同一个GPU上
net[0].weight.data.device

device(type='cuda', index=0)

In [ ]:
# 我们要把数据手动copy到GPU上